# Aula 8 - Pandas Parte III

## <font color="blue"> MBA em Ciência de Dados</font>
## <font color="blue">Programação para Ciência de Dados</font>

### <font color="blue">Pandas Parte III</font>
**Material Produzido por Luis Gustavo Nonato e Bruno Coelho**<br>
**Cemeai - ICMC/USP São Carlos**
---


__Conteúdo:__
- GroupBy
    - Agregação
    - Filtragem
    - Transformação

__Referências__ <br>
- [Pandas: powerful Python data analysis toolkit: Wes McKinney & PyData Devel. Team](https://pandas.pydata.org/pandas-docs/stable/pandas.pdf)
- [http://pandas.pydata.org/pandas-docs/stable/index.html](http://pandas.pydata.org/pandas-docs/stable/index.html)


## Introdução

Frequentemente queremos aplicar transformações e filtragens  em um conjunto de dados a fim de extrair informações relevantes e padrões contidos nos dados. Por exemplo, analisar o salário médio de um grupo de profissionais considerando o sexo e a faixa etária dos profissionais.
Muitas destas transformações podem ser realizadas por meio de uma operação denominada _MapReduce_.

_MapReduce_ é implementada no pacote <font color='blue'>Pandas</font> por meio do método <font color='blue'>groupby</font>.

Em geral, o método <font color='blue'>groupby</font> envolve 3 tarefas:
- __Split__: Divide os dados em subgrupos. Por exemplo, divide os profissionais em subgrupos de acordo com a faixa etária dos profissionais.
- __Apply__: Aplica alguma transformação, agregação ou filtragem para extrair informações de cada subgrupo. Por exemplo, pode-se calcular a média salarial em cada faixa etária dos profissionais.
- __Combine__: Combina os resultados das transformações em um DataFrame ou Série.


$$
\begin{array}{ccccc}
DataFrame & & Split & Apply & Combine\\
\begin{array}{c|c}
C1 & C2 \\ \hline
A & 0 \\ \hline
B & 5 \\ \hline
C & 10 \\ \hline
A & 5 \\ \hline
B & 5 \\ \hline
C & 10 \\ \hline
A & 10 \\ \hline
B & 5 \\ \hline
C & 10 
\end{array} &
\begin{array}{c}
\nearrow \\ \\
\rightarrow \\ \\
\searrow
\end{array} &
\begin{array}{c|c}
A & 0 \\ \hline
A & 5 \\ \hline
A & 10 \\ \\
B & 5 \\ \hline
B & 5 \\ \hline
B & 5 \\ \\
C & 10 \\ \hline
C & 10 \\ \hline
C & 10 
\end{array} &
\begin{array}{c}
\searrow\\ \\
\rightarrow \\ \\
\nearrow
\end{array} & 
\begin{array}{c|c}
A & 15 \\ \hline
B & 15 \\ \hline
C & 30 
\end{array}
\end{array}
$$

A etapa de "split" divide um conjunto de dados de acordo com algum critério, que pode ser valores das colunas do DataFrame, lista de valores externos ou até mesmo o resultado de uma função. 

Na verdade, o  <font color='blue'>pandas</font> não divide o DataFrame, mas cria uma estrutura que permite operar como se os dados estivessem divididos, evitando o uso excessivo de memória.

Portanto, é importante estar atento, pois o resultado de aplicar o método <font color='blue'>groupby</font> não é um novo DataFrame (ou Serie), mas sim um objeto do tipo `groupby`.
Para visualizar ou acessar os grupos criados pode-se realizar uma redução (que é o resultado do "apply" e "combine") ou utilizar algum método do objeto `groupby`, como por exemplo:
- <font color='blue'>first()</font>: apresenta a primeira linha de cada grupo formado
- <font color='blue'>get_group()</font>: retorna um DataFrame com o conteúdo de um grupo
- <font color='blue'>groups()</font>: retorna um dicionário onde as chaves são os rótulos dos grupos e os valores os índices das linhas onde elementos do grupo ocorrem



In [1]:
# Importando o pacote 'pandas' e 'numpy'
import pandas as pd
import numpy as np

# O comando abaixo carrega o arquivo 'tips.csv' disponível no moodle
df = pd.read_csv("tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
# Agrupando os dados de acordo com os valores da coluna "sex"

dfgb_sex = df.groupby("sex")

# Perceba que a variável 'df_sex' não é um DataFrame, mas sim 
# um 'DataFrameGroupBy'
print(type(dfgb_sex))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [3]:
# visualizando a primeira linha de cada grupo
print(type(dfgb_sex.first()))  # note que ao invocar o método first() obtemos um DataFrame
print('\n primeira linha cada grupo:\n',dfgb_sex.first())  # note que os rótulos das linhas são
                                                           # os grupos obtidos

<class 'pandas.core.frame.DataFrame'>

 primeira linha cada grupo:
         total_bill   tip smoker  day    time  size
sex                                               
Female       16.99  1.01     No  Sun  Dinner     2
Male         10.34  1.66     No  Sun  Dinner     3


In [4]:
# visualizando as primeiras linhas do grupo 'Male'
print('\n',type(dfgb_sex.get_group('Male')))  # o resultado do get_group é um DataFrame
print('\n primeiras linhas do grupo "Male":\n',
      dfgb_sex.get_group('Male').head())


 <class 'pandas.core.frame.DataFrame'>

 primeiras linhas do grupo "Male":
    total_bill   tip   sex smoker  day    time  size
1       10.34  1.66  Male     No  Sun  Dinner     3
2       21.01  3.50  Male     No  Sun  Dinner     3
3       23.68  3.31  Male     No  Sun  Dinner     2
5       25.29  4.71  Male     No  Sun  Dinner     4
6        8.77  2.00  Male     No  Sun  Dinner     2


In [5]:
# o método 'groups' retorna um dicionário onde as chaves 
# são os rótulos dos grupos formados e os valores são os índices das linhas
# onde os elementos do grupo estão
print('\n',type(dfgb_sex.groups))  # o resutlado de groups é um dicionário
print('\n rótulos dos grupos:\n',dfgb_sex.groups.keys())  # as chaves são os rótulos dos grupos
print('\n indices de alguns elementos do grupo "Female":\n',
      dfgb_sex.groups['Female'][0:5])


 <class 'pandas.io.formats.printing.PrettyDict'>

 rótulos dos grupos:
 dict_keys(['Female', 'Male'])

 indices de alguns elementos do grupo "Female":
 Int64Index([0, 4, 11, 14, 16], dtype='int64')


#### Agrupando de acordo com uma lista externa
O método <font color='blue'>groupby</font> pode agrupar um DataFrame de acordo com uma lista externa (que não é parte do DataFrame). Para isso, a lista deve possuir um número de elementos igual ao número de linhas do DataFrame.

In [6]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                  'key2': ['one','two','one','two','one'], 
                  'data1': np.random.uniform(low=0,high=1,size=5),
                  'data2': np.random.uniform(low=0,high=1,size=5)})
print(df)

# criando uma lista de 0 e 1 com o mesmo número de linhas do DataFrame
ls = [i for i in np.random.randint(0,2,df.shape[0])]

#lista criada
print(ls) 

  key1 key2     data1     data2
0    a  one  0.563426  0.705356
1    a  two  0.519431  0.742514
2    b  one  0.597513  0.310407
3    b  two  0.822884  0.160701
4    a  one  0.317987  0.582648
[0, 0, 1, 0, 0]


In [7]:
# Agrupando o DataFrame de acordo com a lista 'ls'
gbylist = df.groupby(ls)

print(gbylist.get_group(0))  # linhas onde o valor 0 aparece na lista
print(gbylist.get_group(1))  # linhas onde o valor 1 aparece na lista

  key1 key2     data1     data2
0    a  one  0.563426  0.705356
1    a  two  0.519431  0.742514
3    b  two  0.822884  0.160701
4    a  one  0.317987  0.582648
  key1 key2     data1     data2
2    b  one  0.597513  0.310407


#### Agrupando com múltiplos critérios (indexação hierárquica)
Quando mais que um conjunto de valores é enviado como parâmetro para o <font color='blue'>groupby</font>, o resultado é um agrupamento com índices organizados de forma hierárquica.

In [8]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','a','b','b','a','a','b','a'],
                  'key2': ['one','two','one','two','one','two','two','one'], 
                  'data1': np.random.uniform(low=0,high=1,size=8),
                  'data2': np.random.uniform(low=0,high=1,size=8)})

df

,key1,key2,data1,data2
0,a,one,0.743792,0.633434
1,a,two,0.046428,0.468083
2,b,one,0.648676,0.895709
3,b,two,0.779050,0.220253
4,a,one,0.286265,0.676708
5,a,two,0.595901,0.171790
6,b,two,0.508742,0.738325
7,a,one,0.590149,0.261244


In [9]:
# agrupando com base nos valores das colunas 'key1' e 'key2'
dfh = df.groupby(['key1','key2'])

# imprimindo o conteúdo de cada grupo com um laço for
# 'groupname' corresponde ao índice do grupo, uma tupla neste caso
# 'group' corresponde ao grupo\ própriamente dito
for groupname,group in dfh:
    print('Rotulo do Grupo: ',groupname)
    print(group,'\n')

Rotulo do Grupo:  ('a', 'one')
  key1 key2     data1     data2
0    a  one  0.743792  0.633434
4    a  one  0.286265  0.676708
7    a  one  0.590149  0.261244 

Rotulo do Grupo:  ('a', 'two')
  key1 key2     data1     data2
1    a  two  0.046428  0.468083
5    a  two  0.595901  0.171790 

Rotulo do Grupo:  ('b', 'one')
  key1 key2     data1     data2
2    b  one  0.648676  0.895709 

Rotulo do Grupo:  ('b', 'two')
  key1 key2     data1     data2
3    b  two  0.779050  0.220253
6    b  two  0.508742  0.738325 



#### Agrupando com funções
Uma função pode ser enviada como parâmetro de agrupamento para o <font color='blue'>groupby</font>. Neste caso, a função é aplicada aos rótulos das linhas do DataFrame e o resultado é utilizado como rótulo do grupo.

In [10]:
# constuindo DataFrame com rótulos de linhas e colunas
dfp = pd.DataFrame(data=np.random.randint(low=0, high=10, size=(6,5)),
               columns=['a','b','c','d','e'], 
               index=['Joe','Michel','Steve','Wes','Jim','Travis'])
dfp

,a,b,c,d,e
Joe,3,3,1,0,7
Michel,7,3,8,7,0
Steve,0,2,4,4,8
Wes,1,8,6,4,6
Jim,3,1,6,7,3
Travis,3,2,4,1,7


In [11]:
# agrupando de acordo com o número de caracteres nos rótulos das linhas
gbf = dfp.groupby(lambda x: len(x))

for name, group in gbf:
    print('Grupo: ',name)
    print(group,'\n')

Grupo:  3
     a  b  c  d  e
Joe  3  3  1  0  7
Wes  1  8  6  4  6
Jim  3  1  6  7  3 

Grupo:  5
       a  b  c  d  e
Steve  0  2  4  4  8 

Grupo:  6
        a  b  c  d  e
Michel  7  3  8  7  0
Travis  3  2  4  1  7 



## Transformações e Agregações

Em geral, temos 5 tipos de operações que podem ser aplicadas aos grupos gerados pelo <font color='blue'>groupby</font>:


- __Métodos de agregação__: Combinam várias linhas em um único valor. Exemplos incluem a média, soma e mediana de cada coluna em cada grupo.

- __Métodos de filtragem__: Retornam apenas um subconjunto dos dados originais. 

- __Métodos de tranformação__: Retornam um DataFrame com o mesmo tamanho e índices dos dados originais, mas com valores transformados com base nos grupos.

## Métodos de agregação

Pandas fornece [diversas funções](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) estatísticas de agregação, como <font color='blue'>sum, mean, std, max, min </font>. Pode-se ainda calcular várias agregações simultaneamente com o método <font color='blue'>agg</font> (de "aggregate")

In [12]:
# Importando o pacote 'pandas'
import pandas as pd

# Carregando o arquivo 'tips.csv'
df = pd.read_csv("tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [13]:
# Obtendo a maior gorjeta dos grupos 'Male' e 'Female'
print("Gorjeta máxima")

# Agrupamos por sexo, depois selecionamos a coluna "tip" (gorjeta)
# e pegamos o máximo dela.
print(df.groupby(["sex"])["tip"].max())


Gorjeta máxima
sex
Female     6.5
Male      10.0
Name: tip, dtype: float64


In [14]:
# Pode-se calcular várias agregações simultaneamente via método "agg"
print("Média e Desvio Padrão das gorjetas em cada grupo:")
df.groupby(["sex"])["tip"].agg([ "mean", "std"])

Média e Desvio Padrão das gorjetas em cada grupo:


,mean,std
sex,,
Female,2.833448,1.159495
Male,3.089618,1.489102


**Importante**: O resultado de uma agregação, transformação ou filtragem  é um novo DataFrame onde os rótulos das linhas são os valores utilizados para realizar o agrupamento. 
Caso queiramos um DataFrame com linhas indexadas com valores $0,1,...$, devemos empregar o método <font color='blue'>reset_index</font>.

In [15]:
# Usando as colunas de agregação como índices
result = df.groupby(["sex"])["tip"].agg(["mean", "std"])
print(result,'\n')
print("Rotulos dos grupos:\n", result.index)
print("\nRotulos das colunas:\n", result.columns)

            mean       std
sex                       
Female  2.833448  1.159495
Male    3.089618  1.489102 

Rotulos dos grupos:
 Index(['Female', 'Male'], dtype='object', name='sex')

Rotulos das colunas:
 Index(['mean', 'std'], dtype='object')


In [16]:
# Reindexando com reset_index(), os rótulos se tornam uma nova coluna
result = df.groupby(["sex"])["tip"].agg(["mean", "std"]).reset_index()
print(result,'\n')
print("Rotulos dos grupos:\n", result.index)
print("\nRotulos das colunas:\n", result.columns)

      sex      mean       std
0  Female  2.833448  1.159495
1    Male  3.089618  1.489102 

Rotulos dos grupos:
 RangeIndex(start=0, stop=2, step=1)

Rotulos das colunas:
 Index(['sex', 'mean', 'std'], dtype='object')


Pode-se aplicar agregações diferentes para cada coluna com o uso de dicionário.

In [17]:
print(df.head())
# calculando a média das gorjetas, 
# o valor máximo das refeições 
# e o dia da semana mais frequente (moda) de cada grupo 'Male' e 'Female'
dfm = df.groupby(["sex"]).agg({"tip": "mean", 
                         "total_bill": "max",
                         "day": lambda x: x.mode()})


print('\n',dfm.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

              tip  total_bill   day
sex                               
Female  2.833448       44.30  Thur
Male    3.089618       50.81   Sat


In [18]:
# Pode-se renomear as colunas do DataFrame gerado durante a agregação
# renomeando colunas com agg
print(df.groupby(["sex"]).agg(mean_tip=("tip", "mean"),
                              max_bill=("total_bill", "max"),
                              most_freq_day=("day", lambda x: x.mode())))

        mean_tip  max_bill most_freq_day
sex                                     
Female  2.833448     44.30          Thur
Male    3.089618     50.81           Sat


## Métodos de filtragem

O método mais comum de filtragem é o <font color='blue'>filter </font>que retorna os elementos do grupo que satisfaz uma condição dada. 

In [19]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.824008,0.062613,0.154448
1,c,0.957235,0.567239,0.606426
2,b,0.786055,0.336015,0.589647
3,c,0.920663,0.929276,0.484257
4,a,0.090002,0.864007,0.113029
5,a,0.274986,0.706786,0.416058
6,b,0.111060,0.755031,0.251001
7,a,0.349469,0.317557,0.404267
8,c,0.729898,0.317614,0.175853
9,b,0.965698,0.773362,0.252772


In [20]:
# média geral da coluna 'data1'
data1_mean = df['data1'].mean()
print('media da coluna "data1"\n',data1_mean)

# calculando a média de cada coluna em cada grupo dado pela coluna 'key1'
print('\nmedia de cada grupo\n',df.groupby('key1').mean())

# filtrando os grupos cuja média da coluna 'data1' é maior que a média 
# geral da coluna 'data1'
print('\n grupos cuja média da coluna "data1" é maior que a média geral de "data1"\n')
print(df.groupby('key1').filter(lambda x : x['data1'].mean() > data1_mean))

media da coluna "data1"
 0.5629499966826673

media de cada grupo
          data0     data1     data2
key1                              
a     0.384616  0.487741  0.271950
b     0.620937  0.621469  0.364473
c     0.869266  0.604710  0.422179

 grupos cuja média da coluna "data1" é maior que a média geral de "data1"

  key1     data0     data1     data2
1    c  0.957235  0.567239  0.606426
2    b  0.786055  0.336015  0.589647
3    c  0.920663  0.929276  0.484257
6    b  0.111060  0.755031  0.251001
8    c  0.729898  0.317614  0.175853
9    b  0.965698  0.773362  0.252772


## Métodos de transformação

O método <font color='blue'>transform</font> retorna um DataFrame com o **mesmo número de linhas** que o DataFrame original. A transformação é realizada em cada elemento de cada grupo.

In [21]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.263425,0.637179,0.589697
1,c,0.397369,0.335197,0.681486
2,b,0.553715,0.250478,0.040710
3,c,0.300760,0.266932,0.027358
4,a,0.467385,0.921366,0.254093
5,a,0.911012,0.651333,0.300607
6,b,0.491445,0.459021,0.467461
7,a,0.212382,0.162964,0.028618
8,c,0.946063,0.354482,0.563226
9,b,0.789423,0.089889,0.220920


In [22]:
# Calcula a diferença entre a média do grupo e 
# o elemento do grupo (em cada coluna)
print('média de cada grupo\n',df.groupby("key1").mean())

df[['d1','d2','d3']] = df.groupby("key1").transform(lambda x: x-x.mean())
print('\n',df)

média de cada grupo
          data0     data1     data2
key1                              
a     0.463551  0.593210  0.293254
b     0.611528  0.266463  0.243030
c     0.548064  0.318870  0.424023

   key1     data0     data1     data2        d1        d2        d3
0    a  0.263425  0.637179  0.589697 -0.200126  0.043968  0.296444
1    c  0.397369  0.335197  0.681486 -0.150695  0.016326  0.257463
2    b  0.553715  0.250478  0.040710 -0.057812 -0.015985 -0.202320
3    c  0.300760  0.266932  0.027358 -0.247304 -0.051938 -0.396665
4    a  0.467385  0.921366  0.254093  0.003834  0.328156 -0.039161
5    a  0.911012  0.651333  0.300607  0.447461  0.058123  0.007353
6    b  0.491445  0.459021  0.467461 -0.120083  0.192558  0.224431
7    a  0.212382  0.162964  0.028618 -0.251169 -0.430247 -0.264636
8    c  0.946063  0.354482  0.563226  0.397999  0.035611  0.139203
9    b  0.789423  0.089889  0.220920  0.177895 -0.176574 -0.022110


O método <font color='blue'>apply</font> aplica uma função em cada grupo e retorna um DataFrame com o resultado da função em cada grupo. Os rótulos das linhas do DataFrame gerado são os identificadores dos grupos. O método <font color='blue'>apply</font> é o mais versátil dos métodos de transformação.

In [23]:
# Construindo DataFrame a partir de um dicionário
df = pd.DataFrame({'key1': ['a','c','b','c','a','a','b','a','c','b'],
                  'data0': np.random.uniform(low=0,high=1,size=10),
                  'data1': np.random.uniform(low=0,high=1,size=10),
                  'data2': np.random.uniform(low=0,high=1,size=10)})

df

,key1,data0,data1,data2
0,a,0.283507,0.138905,0.983034
1,c,0.929546,0.504444,0.091691
2,b,0.796047,0.310626,0.958080
3,c,0.207093,0.230635,0.691981
4,a,0.939277,0.688353,0.814402
5,a,0.697589,0.255067,0.733194
6,b,0.046932,0.222266,0.561578
7,a,0.468746,0.633056,0.493251
8,c,0.169524,0.654275,0.149334
9,b,0.780990,0.562966,0.837054


In [24]:
# agrupando pela coluna 'key1' e calculando a 
# diferença entre o maior e o menor valor de cada coluna em cada grupo
# e retornando a menor diferença

# diferença entre o maior e menor valor em cada coluna em cada grupo
print(df.groupby('key1').apply(lambda x: (x.max()-x.min())))

# menor diferença em cada grupo
max_min = df.groupby('key1').apply(lambda x: (x.max()-x.min()).sort_values()[0])
print(max_min)

         data0     data1     data2
key1                              
a     0.655769  0.549448  0.489782
b     0.749115  0.340700  0.396502
c     0.760021  0.423640  0.600289
key1
a    0.489782
b    0.340700
c    0.423640
dtype: float64


**Comentário: MapReduce**

Embora frequentemente chamamos qualquer operação split-apply-combine de _MapReduce_, originalmente esse nome foi dado ao modelo de programação desenvolvido pela Apache.

Sua funcionalidade é a mesma do _GroupBy_ do Pandas, porém foi implementado para operar em grandes volumes de dados utilizando algoritmos distribuidos em uma arquitetura de computação paralela (cluster de computadores), garantindo que cada máquina opere sobre um subconjunto específico dos dados, sendo o padrão para tecnologias de Big Data como [Hadoop](https://hadoop.apache.org/)
